In [9]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.11:2.1.0,com.databricks:spark-avro_2.11:3.2.0 pyspark-shell'

In [10]:
from pytwitter import StreamApi
import json
import os
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 


In [11]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pykafka import KafkaClient

In [12]:
client = KafkaClient()

In [13]:
producer = client.topics["test1"].get_producer()

In [22]:
class MyStream(StreamApi):
    def __init__(self, bearer_token):
        try:
            StreamApi.__init__(self, bearer_token=bearer_token)
            self.__file_index = 0
        except KeyError as e:
            print("error on_tweet: %s" % str(e))

    def on_data(self, raw_data, return_json=False):
        # print(tweet)
        # tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)
        # with open(tweets_file_path, 'w', encoding='utf-8') as outfile:
        #         json.dump(tweet, outfile, ensure_ascii=False)
        # while os.path.exists(tweets_file_path) and os.stat(tweets_file_path).st_size > 2**10:
        #     self.__file_index += 1 
        #     tweets_file_path = 'tweets_json/{}.json'.format(self.__file_index)
        data = json.loads(raw_data)
        print(data)
        tweet_info = {}
        try:
            tweet_info["tweet"] = data["data"]["text"]
            tweet_info["created_at"] = data["data"]["created_at"]
            print(tweet_info)
            producer.produce(bytes(json.dumps(tweet_info), "utf-8"))
        except KeyError as e:
            print("error on_tweet: %s" % str(e))

            

In [15]:
rules = {
        "add": [
            {"value": "cat has:media", "tag": "cats with media"},
            {"value": "cat has:media -grumpy", "tag": "happy cats with media"}
        ]
     }

In [23]:
stream = MyStream(bearer_token=credentials.BEARER_TOKEN)
stream.manage_rules(rules=rules)
stream.search_stream(tweet_fields = ("created_at"))
# stream.sample_stream(return_json=True)

{'data': {'created_at': '2022-09-25T17:24:37.000Z', 'id': '1574087489748287490', 'text': 'RT @meow_utopia: Cat Bunny\n\n#Caturday https://t.co/4JkFrjJ0Tj'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
{'tweet': 'RT @meow_utopia: Cat Bunny\n\n#Caturday https://t.co/4JkFrjJ0Tj', 'created_at': '2022-09-25T17:24:37.000Z'}
{'data': {'created_at': '2022-09-25T17:24:38.000Z', 'id': '1574087494206951428', 'text': 'RT @cat_dot_exe: https://t.co/t4jRiOgntm'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, {'id': '1545037781386153987', 'tag': 'cats with media'}]}
{'tweet': 'RT @cat_dot_exe: https://t.co/t4jRiOgntm', 'created_at': '2022-09-25T17:24:38.000Z'}
{'data': {'created_at': '2022-09-25T17:24:37.000Z', 'id': '1574087491279282179', 'text': 'RT @cat_auras: https://t.co/fgNzOpHFbG'}, 'matching_rules': [{'id': '1545037781386153986', 'tag': 'happy cats with media'}, 

KeyboardInterrupt: 